In [32]:
pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\Мишаня\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



     -------------------------------------- 250.0/250.0 kB 2.6 MB/s eta 0:00:00


In [1]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 78.6 MB/s eta 0:00:00


In [2]:
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pulp import *
import numpy as np
import pandas as pd

вектор-строка = характеристика ВРЕМЕНИ (почасовая в рамках одной электростанции)

вектор-столбец = характеристика ЭЛЕКТРОСТАНЦИИ (по электростанциям в рамках часа)

array[j,i] - j - номер электростанции, i - номер часа
    
    
    Итерация по i = итерация по часам
    Итерация по j = итерация по электростанциям

In [28]:
class LPmodel:
    # Задаем объекты класса
    def __init__(self, p_demand, stations_df, VRE_df='None', HPS_df='None'):
        self.p_demand = p_demand  # потребление эл-гии в течение одного года (час) МВТч, вектор-строка
        self.stations_df = stations_df
        self.VRE_df = VRE_df
        self.HPS_df = HPS_df
        self.p_nonVRE_cap = self.stations_df[:,[1]]  # Nуст для НеВИЭ, вектор-столбец
        # Определение задачи оптимизации
        self.model = LpProblem("Energy_System_Optimization", LpMinimize)
        # Переменные избытка/недостатка мощности
        self.v_curtail = np.array([])
        self.v_loss_of_load = np.array([])
        # Переменные мощностей
        self.v_online = np.array([])
        self.v_use_online = np.array([])
        self.v_gen_capacity = np.array([])
        self.v_reserve_capacity = np.array([])
        # Переменные - №1 пок-ет только значения возрастающих, №2 пок-ет модуль изменения нагрузок
        self.v_startup = np.array([])
        self.v_abs_ramp = np.array([])
        # Переменные для ГЭС
        if isinstance(self.HPS_df, str) == False:  # проверка на тип данных
            self.v_outflow_HPS = np.array([])
            self.v_storage_HPS_outflow = np.array([])
        else:
            self.v_outflow_HPS = np.array([0 for i in range(len(self.p_demand))])

    # метод для создания матриц - переменных
    def __matrix_variable(self, name: str, customize_i = 0, customize_j = 0, **kwargs):
        return np.array([[LpVariable(name + str(j) + '_' + str(i), **kwargs)\
                                for i in range(len(self.p_demand) + customize_i)] 
                                for j in range(len(self.p_nonVRE_cap) + customize_j)])
    
    def __vector_variable(self, name: str, direction = 'horizontal', customize = 0, **kwargs):
        if direction == 'horizontal':
            return np.array([LpVariable(name + str(i), **kwargs)\
                                        for i in range(len(self.p_demand) + customize)])
        elif direction == 'vertical':
            return np.array([[LpVariable(name + str(i), **kwargs)]\
                                        for i in range(len(self.p_demand) + customize)])
        else:
            raise ValueError('direction может быть только "vertical" или "horizontal" !!!')
            
    # скрытый метод, вызываемый при переданном VRE_df
    def __VRE_constraint(self):
        # Параметры ВИЭ
        p_cf_vre = self.VRE_df[:,2]  # КИУМ почасовой для ВИЭ, вектор-строка
        p_VRE_capacity = self.VRE_df[0,0]  # Nуст ВИЭ
        p_omCost_VRE = self.VRE_df[0,1]  # Переменные затраты для ВИЭ
        p_gen_VRE_capacity = p_cf_vre * p_VRE_capacity #мощность ВИЭ
        return (p_gen_VRE_capacity, p_omCost_VRE)
    
    # скрытый метод, вызываемый при переданном HPS_df
    def __HPS_constraint(self):
        # Параметры ГЭС
        p_inflow_HPS = self.HPS_df[:,3] # Приток воды на ГЭС, МВт*ч
        p_storage_HPS_max = self.HPS_df[0,0] # Максимальная ёмкость водохранилища ГЭС, МВт
        p_storage_HPS_start = self.HPS_df[0,1] # Ёмкость водохранилища ГЭС в первый час, МВт
        p_max_water_flow = self.HPS_df[0,2] # Максимальный пропуск воды, МВт*ч
        
        # Переменные для ГЭС
        self.v_outflow_HPS = self.__vector_variable('v_outflow_HPS_', direction='horizontal', lowBound = 0, upBound = p_max_water_flow)
        self.v_storage_HPS_outflow = self.__vector_variable('v_storage_HPS_outflow_', direction='horizontal',
                                                            lowBound = 0, upBound = p_storage_HPS_max)  # Ёмкость хранилища, исп-я для генерации, МВт
        # Ограничения на ГЭС
        for i in range(len(self.p_demand)):
            self.model += self.v_outflow_HPS[i] <= self.v_storage_HPS_outflow[i]
            if i != 0:
                self.model += self.v_storage_HPS_outflow[i] == self.v_storage_HPS_outflow[i-1] + p_inflow_HPS[i-1] - self.v_outflow_HPS[i-1]
            else:
                self.model += self.v_storage_HPS_outflow [i] == p_storage_HPS_start
        return ''
    
    def solve_it(self):
        # Параметры неВИЭ
        p_cf_reserve = self.stations_df[:,[2]]  # Коэффициент резерва
        p_nonVRE_minload = self.stations_df[:,[3]]  # Минимальная нагрузка для НеВИЭ
        p_ramp_up = self.stations_df[:,[4]]
        p_ramp_down = self.stations_df[:,[5]]
        
        # Параметры баланса (КОНСТАНТЫ ВРЕМЕННО)
        p_ts_reserve_node = 25  # почасовой резерв для заданных узлов
        p_transfer = 10  # объем передачи эл-гии между узлами
        p_convert = 0  # объем конвертации энергии
        p_charge_discharge = 15  # зарядка/разрядка хранилищ
        p_import_export = np.array([0,0,0,0])  # импорт - экспорт вне модели (из-за границы)
        p_ts_import = np.array([0,0,0,0])  # почасовой импорт-экспорт (ноды)
        
        # Параметры экономического функционала
        p_omCost_nonVRE = self.stations_df[:,[6]]  # Переменные затраты НеВИЭ 
        p_eff = self.stations_df[:,[7]]  # эффективность КПД, вектор-столбец
        p_fuel_cost = self.stations_df[:,[8]]  # топливные затраты, вектор-столбец
        p_loss_of_load_penalty = 5000  # штраф за потерю нагрузку, CUR/MWh
        p_curtailment_VRE_penalty = 2000  # штраф на певышение ВИЭ
        p_reserves_penalty = 1000  # штраф за превышение резервов
        p_startup_cost = self.stations_df[:,[9]]  # затраты за запуск доп. мощности, руб/Mwh
        
        # Получаем значения от ВИЭ, если их передали
        if isinstance(self.VRE_df, str) == False:  # проверка на тип данных:
            VRE_attributes = self.__VRE_constraint()
            p_gen_VRE_capacity = VRE_attributes[0]
            p_omCost_VRE = VRE_attributes[1]
        else:
            p_gen_VRE_capacity = np.array([0 for i in range(len(self.p_demand))])
            p_omCost_VRE = 0
            
        # Добавляем ограничения от ГЭС, если их передавали
        if isinstance(self.HPS_df, str) == False:  # проверка на тип данных:
            self.__HPS_constraint()
        
        # Переменные избытка/недостатка мощности
        self.v_curtail = self.__vector_variable('v_curtail_', direction='horizontal', lowBound = 0)  # избыток генерации ВИЭ
        self.v_loss_of_load = self.__vector_variable('v_loss_of_load_', direction='horizontal', lowBound = 0)  # недостаток генерации

        # Переменные мощностей
        self.v_online = self.__matrix_variable('v_online_', lowBound = 0)  # онлайн - доступная мощность
        self.v_use_online = self.__matrix_variable('v_use_online_', cat = 'Binary')  # параметр работы станции
        self.v_gen_capacity = self.__matrix_variable('v_gen_capacity_', lowBound = 0)  # мощность на генерации
        self.v_reserve_capacity = self.__matrix_variable('v_reserve_capacity_', lowBound = 0)  # мощность обяз-го резерва
        
        # Переменные - №1 пок-ет только значения возрастающих, №2 пок-ет модуль изменения всех нагрузок
        self.v_startup = self.__matrix_variable('v_startup_', customize_i=-1, lowBound = 0)
        self.v_abs_ramp = self.__matrix_variable('v_abs_ramp_', lowBound = 0)
        
        # Ограничения по онлайн, генерации и резервам
        for i in range(len(self.p_demand)):
            for j in range(len(self.p_nonVRE_cap)):
                self.model += (self.v_use_online * self.p_nonVRE_cap)[j,i] == self.v_online[j,i]
                self.model += self.v_online[j,i] <= self.p_nonVRE_cap[j,0]
                self.model += p_cf_reserve[j,0]*self.v_online[j,i] >= self.v_reserve_capacity [j,i]  # обязательный резерв
                self.model += self.v_gen_capacity [j,i] + self.v_reserve_capacity [j,i] <= self.v_online [j,i]
                self.model += self.v_gen_capacity[j,i] >= self.v_online[j,i]*p_nonVRE_minload[j,0]  # минимальная нагрузка
                if i != 0:  # ограничения на время набора/сброса нагрузки
                    self.model += self.v_gen_capacity[j,i] - self.v_gen_capacity[j,i-1] <= self.p_nonVRE_cap[j,0]*p_ramp_up[j,0] 
                    self.model += self.v_gen_capacity[j,i-1] - self.v_gen_capacity[j,i] <= self.p_nonVRE_cap[j,0]*p_ramp_down[j,0] 
                    # подсчет модуля изменений для нахождения отдельных генераций с ростом
                    self.model += self.v_abs_ramp[j,i-1] >= self.v_gen_capacity[j,i] - self.v_gen_capacity[j,i-1]
                    self.model += self.v_abs_ramp[j,i-1] >= self.v_gen_capacity[j,i-1] - self.v_gen_capacity[j,i]
                    self.model += self.v_startup[j,i-1] == (self.v_abs_ramp[j,i-1] + self.v_gen_capacity[j,i] - \
                                           self.v_gen_capacity[j,i-1])/2

        # Уравнение спрос = предложение
        for i in range(len(self.p_demand)):  
            # часть уравнения по генерации
            self.model += sum(self.v_gen_capacity[:,i]) + p_gen_VRE_capacity[i] - self.v_curtail[i]\
                        + p_import_export[i] + self.v_loss_of_load[i] + self.v_outflow_HPS[i]\
            \
                    == self.p_demand[i] + p_ts_import[i] + p_ts_reserve_node + p_transfer + p_convert\
                        + p_charge_discharge
            # часть уравнения по спросу

        # Ограничение на ВИЭ
        for i in range(len(self.p_demand)): 
            self.model += self.v_curtail [i] <= p_gen_VRE_capacity[i]  

        # Экономическая целевая функция (Фунционал)
        v_obj = sum(p_omCost_nonVRE * self.v_gen_capacity)\
            + p_omCost_VRE * sum(p_gen_VRE_capacity)\
            + sum(self.v_gen_capacity * (1/p_eff) * p_fuel_cost)\
            + p_loss_of_load_penalty * sum(self.v_loss_of_load) \
            + p_curtailment_VRE_penalty * sum(self.v_curtail)\
            + p_reserves_penalty * sum(p_cf_reserve*self.v_online-self.v_reserve_capacity)\
            + sum(sum(p_startup_cost * self.v_startup))
        self.model += lpSum(v_obj)
        # Решаем задачу оптимизации
        return self.model.solve()
    
    # метод для вывода результатов в командной строке
    def print_result(self):
        for i in range(len(self.p_demand)):
            print(f"Optimal v_curtail_{i}: ", self.v_curtail[i].value())
            print(f"Optimal v_loss_of_load_{i}: ", self.v_loss_of_load[i].value())
            for j in range(len(self.p_nonVRE_cap)):
                print(f"Optimal v_gen_capacity_{j}_{i}h: ", self.v_gen_capacity[j,i].value())
            if isinstance(self.HPS_df, str) == False:
                print(f"Storage of HPS_{i}: " ,self.v_storage_HPS_outflow[i].value())
                print(f"HPS gen_{i}: " ,(self.v_outflow_HPS[i].value()))
            print()
        # показатель v_online по часам - 0 если элеткростанция не работает, 
                                        #50 (из self.p_nonVRE_cap) - мощность станции в работе
        for i in range(len(self.p_demand)):
            for j in range(len(self.p_nonVRE_cap)):
                print(self.v_online[j,i].value())
            print()
    
    # метод для запрашивания матрицы результатов
    def get_matrix(self):
        result_list = [['Час'] + self.stations_df[:,0].tolist() + ['Перевыработка', 'Недостаток нагрузки']]
        if isinstance(self.HPS_df, str) == False:
            result_list[0].append('Водохранилище')
            result_list[0].append('Выработка ГЭС')
        for i in range(len(self.p_demand)):
            temp_list = []
            temp_list.append(i)
            for j in range(len(self.p_nonVRE_cap)):
                temp_list.append(self.v_gen_capacity[j,i].value())
            temp_list.append(self.v_curtail[i].value())
            temp_list.append(self.v_loss_of_load[i].value())
            if isinstance(self.HPS_df, str) == False:
                temp_list.append(self.v_storage_HPS_outflow[i].value())
                temp_list.append(self.v_outflow_HPS[i].value())
            result_list.append(temp_list)
        return pd.DataFrame(np.array(result_list).T)

In [9]:
# Путь к файлам для сбора данных
demand_file = './Model_files/demand.xlsx'
stations_file = './Model_files/Stations.xlsx'
VRE_file = './Model_files/VRE.xlsx'
HPS_file = './Model_files/HPS.xlsx'

save_file = './Model_files/result.xlsx'

In [29]:
# Открывание файлов и перевод их в матрицы NumPy
stations_data = pd.read_excel(stations_file, index_col = 0).to_numpy()
demand_data = pd.read_excel(demand_file).to_numpy()[:,1]
VRE_data = pd.read_excel(VRE_file, index_col=0).to_numpy()
HPS_data= pd.read_excel(HPS_file, index_col=0).to_numpy()

my_model = LPmodel(demand_data, stations_data, HPS_df=HPS_data, VRE_df=VRE_data)
my_model.solve_it()

1

In [30]:
my_model.get_matrix()

,0,1,2,3,4
0,Час,0,1,2,3
1,Угольные станции,25.0,25.0,50.0,50.0
2,АЭС,0.0,17.5,42.5,42.5
3,ПГУ,0.0,25.0,50.0,50.0
4,ПТУ,0.0,17.5,42.5,42.5
5,Перевыработка,0.0,40.0,0.0,0.0
6,Недостаток нагрузки,0.0,0.0,0.0,0.0
7,Водохранилище,500.0,495.0,505.0,500.0
8,Выработка ГЭС,15.0,0.0,15.0,15.0


In [13]:
# метод get_matrix() выводит готовый результат в виде DataFrame 
my_model.get_matrix()

,0,1,2,3,4
0,Час,0,1,2,3
1,Угольные станции,25.0,25.0,50.0,50.0
2,АЭС,0.0,17.5,42.5,42.5
3,ПГУ,0.0,25.0,50.0,50.0
4,ПТУ,0.0,17.5,42.5,42.5
5,Перевыработка,0.0,40.0,0.0,0.0
6,Недостаток нагрузки,0.0,0.0,0.0,0.0
7,Водохранилище,500.0,495.0,505.0,500.0
8,Выработка ГЭС,15.0,0.0,15.0,15.0


In [9]:
# сохранение DataFrame с результатами в эксель
my_model.get_matrix().to_excel(save_file)